# Diving more into Hist

In [ ]:
# Make the necessary imports
from hist import Hist
import hist
import numpy as np
import matplotlib.pyplot as plt

hist supports the whole workflow for a histogram's lifecycle, including some plotting tools and shortcuts which are pretty useful for HEP studies. Here, you can see how to serialize/deserialize (will be achieved), construct, use, and visualize histograms.

![histogram's lifecycle](https://tva1.sinaimg.cn/large/007S8ZIlgy1ggrgi6xk7fj30y108qjsf.jpg)

## ND Histograms - More dimensions

The same API works for multiple dimensions.

In [ ]:
## define and fill the histogram
h = (
    Hist.new.Reg(50, -5, 5, name="S", label="s [units]", flow=False)
    .Reg(50, -5, 5, name="W", label="w [units]", flow=False)
    .Double()
)

s_data = np.random.normal(size=50_000)
w_data = np.random.normal(size=50_000)

# fill
h.fill(W=w_data, S=s_data)

We can access and perform various functions on the bins, let's get our hands on them. Can you make a guess?

<details><summary>Click to show answer</summary>

```python
# access by bin number
h[25, 25]

# access by data coordinate
# identical to: h[hist.loc(0), hist.loc(0)]
h[0j, 0j]

# identical to: h[hist.loc(-1) + 5, hist.loc(-4) + 20]
h[-1j + 5, -4j + 20]

# identical to: h.project("S")[20 : 30 : hist.rebin(2)]
h.project("S")[20:30:2j]
```

</details>

In [ ]:
# enter the answer here

Dictionary is allowed when accessing bins. If you have axes all with names in your Hist, you can also access them according to the axes' names.

Let's try a 3D histogram

In [ ]:
data3d = [np.random.normal(size=1_000_000) for _ in range(3)]

hist3d = hist.Hist(
    hist.axis.Regular(150, -5, 5),
    hist.axis.Regular(100, -5, 5),
    hist.axis.Regular(100, -5, 5)
)

hist3d.fill(*data3d)

Let's project to the first two axes:

In [ ]:
hist3d.project(0,1).plot();

Maybe a 5D histogram now?

In [ ]:
s = Hist(
    hist.axis.Regular(50, -5, 5, name="Norm", label="normal distribution"),
    hist.axis.Regular(50, -5, 5, name="Unif", label="uniform distribution"),
    hist.axis.StrCategory(["hi", "hello"], name="Greet"),
    hist.axis.Boolean(name="Yes"),
    hist.axis.Integer(0, 1000, name="Int"),
)

In [ ]:
s.fill(
    Norm=np.random.normal(size=1000),
    Unif=np.random.uniform(size=1000),
    Greet=["hi"] * 800 + ["hello"] * 200,
    Yes=[True] * 600 + [False] * 400,
    Int=np.ones(1000),
)

And we can ofcourse perform similar operations. :)

In [ ]:
s[0j, -0j + 2, "hi", True, 1]

In [ ]:
s[{0: 0j, 3: True, 4: 1, 1: -0j + 2, 2: "hi"}] = 10

s[{"Greet": "hi", "Unif": -0j + 2, "Yes": True, "Int": 1, "Norm": 0j}]

### Get Density

In [ ]:
h[25:30, 25:30].density()

## 2.2: UHI

Let's explore the boost-histogram UHI syntax. We will reuse the previous 2D histogram from part 3:

In [ ]:
h.plot();

I can see that I want y from 0.25 to 0.75, in data coordinates:

In [ ]:
h[:, .25j:.75j].plot();

What's the contents of a bin?

In [ ]:
hist3[100,87]

How about in data coordinates?

In [ ]:
hist3[.5j, .75j]

> Note: to get the coordinates manually:
> 
> ```python
hist3.axes[0].index(.5) == 100
hist3.axes[1].index(.75) == 87
```

How about a 1d histogram?

In [ ]:
hist3[:, ::sum].plot();
hist3[::sum, :].plot();

Let's look at one part and rebin:

In [ ]:
hist3[:50:2j, 50::2j].plot();

What is the value at `(-.75, .5)`?

In [ ]:
hist3[-.75j, .5j]